In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os 
import pandas as pd
%matplotlib inline
import seaborn as sns
import sklearn

In [2]:
cars = pd.read_csv("dataframe_NoIndex_YesHeader-_C.csv")
# print(cars.head())

In [3]:
def countDigit(n):
    count = 0
    while n >= 1:
        n //= 10
        count += 1
    return count

In [4]:
cars.loc[cars['currency'] == 0, 'price'] = cars['price']/3.74901
cars.loc[cars['currency'] == 1, 'price'] = cars['price']/3.67264
cars.loc[cars['currency'] == 2, 'price'] = cars['price']/3.63179
cars.loc[cars['currency'] == 3, 'price'] = cars['price']/0.30191
cars.loc[cars['currency'] == 4, 'price'] = cars['price']/0.38394
cars.loc[cars['currency'] == 5, 'price'] = cars['price']/0.37489
cars['price'] = cars['price'].astype(float)
# print(cars.info())
# print(cars['price'], cars['currency'])


#Need to fix length, width, and height that are larger than 3 digits.
for row in range(cars.shape[0]):
    if countDigit(cars['Length'][row]) >= 3:
        cars['Length'][row] = cars['Length'][row]/(10**(countDigit(cars['Length'][row])-1))
    if countDigit(cars['Width'][row]) >= 3:
        cars['Width'][row] = cars['Width'][row]/(10**(countDigit(cars['Width'][row])-1))
    if countDigit(cars['Height'][row]) >= 3:
        cars['Height'][row] = cars['Height'][row]/(10**(countDigit(cars['Height'][row])-1))
    if countDigit(cars['Wheelbase'][row]) >= 3:
        cars['Wheelbase'][row] = cars['Wheelbase'][row]/(10**(countDigit(cars['Wheelbase'][row])-1))

C:\Users\bacch\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
C:\Users\bacch\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\bacch\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\bacch\Anaconda3\lib\site-packages\ipykernel_launcher.

In [5]:
#Droping duplicates
from scipy import stats
cars = cars.drop_duplicates(subset=['name', 'Country', 'price'], keep='first')
cars = cars.drop(columns=['name'])

#Removing outliers
cars = cars[(np.abs(stats.zscore(cars)) < 3).all(axis=1)]
# print(cars.shape)

In [6]:
cars['Volume'] = cars['Length'] * cars['Width'] * cars['Height']

In [7]:
cars = cars.drop(columns=['Length', 'Width', 'Height', 'Wheelbase'])

In [8]:
# cars.head()

In [9]:
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()
X_col = cars
X_col = X_col.drop(['price', 'Country', 'currency'], axis=1)
X_scaled = scaler.fit_transform(X_col.values)
# print(X_scaled)
X_scaled = pd.DataFrame(X_scaled, columns = ['Engine Capacity', 'Cylinders', 'Drive Type', 'Fuel Tank Capacity',
            'Fuel Economy', 'Fuel Type', 'Horsepower', 'Torque', 
            'Transmission', 'Top Speed', 'Seating Capacity', 'Acceleration', 
            'Trunk Capacity','Volume'])


#Important Predictors
#Engine Capacity, Cylinders, Horsepower, Torque, Top Speed, Acceleration
# X_scaled = X_scaled.drop(['Drive Type', 'Fuel Tank Capacity','Fuel Economy', 'Fuel Type','Transmission','Seating Capacity','Trunk Capacity','Volume'], axis=1)

In [10]:
#Create random training and testing sets
from mlxtend.preprocessing import shuffle_arrays_unison
from sklearn.model_selection import train_test_split

X=X_scaled[['Engine Capacity', 'Cylinders','Horsepower', 'Torque','Top Speed', 'Acceleration']]
y=cars[['price']]
y = np.sqrt(y)

X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.25,random_state=18)

X_train_arr = X_train.values
X_test_arr = X_test.values
y_train_arr = y_train.values
y_test_arr = y_test.values

In [11]:
#Linear Regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from mlxtend.evaluate import bias_variance_decomp
Linear_model = LinearRegression().fit(X_train, y_train)
y_pred = Linear_model.predict(X_test)

# print(Linear_model.coef_)

print("Linear Regression")
print("R-squared = ", Linear_model.score(X_test, y_test))
print("MAE = ",mean_absolute_error(y_test, y_pred))
print("MSE = ", mean_squared_error(y_test, y_pred))

Linear Regression
R-squared =  0.850819714493983
MAE =  24.88854276657499
MSE =  1190.2838735285727


In [12]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score
poly = PolynomialFeatures(degree = 3)
X_poly = poly.fit_transform(X_train)

poly.fit(X_poly, y_train)
poly_model = LinearRegression().fit(X_poly, y_train)
X_poly_test = poly.fit_transform(X_test)

y_pred = poly_model.predict(X_poly_test)

print("Polynomial Regression: degree = 3")
print("R-squared = ", r2_score(y_test, y_pred))
print("MAE = ",mean_absolute_error(y_test, y_pred))
print("MSE = ", mean_squared_error(y_test, y_pred))

Polynomial Regression: degree = 3
R-squared =  0.8778335521024299
MAE =  21.384771838540555
MSE =  974.7451033862058


In [40]:
X_poly_test = poly.fit_transform(np.array([[3.01], [0], [0], [0], [0], [0]]))

y_pred = poly_model.predict(X_poly_test)
print(y_pred)

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 84 is different from 4)

In [13]:
# #Show models for each predictor in linear just to give an idea. 
# for p in ['Engine Capacity', 'Cylinders','Horsepower', 'Torque','Top Speed', 'Acceleration']:
# #     Xp = X_train[p].to_numpy().reshape(-1, 1)
# #     Xp_test = X_test[p].to_numpy().reshape(-1, 1)
# #     model = LinearRegression().fit(Xp, y_train)
# #     y_pred = Linear_model.predict(Xp_test)
    
#     fig = plt.figure(figsize=(8,5))
#     sns.regplot(x = cars[p], y = cars['price'])
# #     x = np.linspace(min(cars[p]), max(cars[p]), 100)
# #     y = model.intercept_ + model.coef_ * x
# #     y = y.reshape(-1,1)
# #     plt.plot(x, y, '-r', label='Linear Model')
#     plt.xlabel(p, size=18)
#     plt.ylabel('Price', size=18);

In [14]:
#Show models for each predictor in polynomial
# for p in ['Engine Capacity', 'Cylinders','Horsepower', 'Torque','Top Speed', 'Acceleration']:
    

In [15]:
#Everything below was for Phase 2. Just uncomment and run

In [16]:
# from yellowbrick.regressor import ResidualsPlot

# visualizer = ResidualsPlot(Linear_model, hist=False)

# visualizer.fit(X_train, y_train)  
# visualizer.score(X_test, y_test)  
# visualizer.show()



# # y_pred = pd.DataFrame(y_pred, columns = ['price'])
# residuals = y_test - y_pred
# sns.histplot(residuals, kde=True)
# plt.title("Residual Count")
# plt.show()

# plt.scatter(y_test, y_pred)
# plt.title('Actual vs Predicted Values')
# plt.xlabel('Actual')
# plt.ylabel('Predicted')
# plt.show()


In [17]:
# #Polynomial Regression
# #Create a model for each important predictor
# from sklearn.preprocessing import PolynomialFeatures
# from sklearn.metrics import r2_score
# poly = PolynomialFeatures(degree = 2)
# X_poly = poly.fit_transform(X_train)

# # print(X_poly.shape)

# poly.fit(X_poly, y_train)
# poly_model = LinearRegression().fit(X_poly, y_train)
# X_poly_test = poly.fit_transform(X_test)

# y_pred = poly_model.predict(X_poly_test)

# print("Polynomial Regression: degree = 2")
# print("R-squared = ", r2_score(y_test, y_pred))
# print("MAE = ",mean_absolute_error(y_test, y_pred))
# print("MSE = ", mean_squared_error(y_test, y_pred))

In [18]:
# visualizer = ResidualsPlot(poly_model, hist=False)

# visualizer.fit(X_poly, y_train)  
# visualizer.score(X_poly_test, y_test)  
# visualizer.show()

# # y_pred = pd.DataFrame(y_pred, columns = ['price'])
# residuals = y_test - y_pred
# sns.histplot(residuals, kde=True)
# plt.title("Residual Count")
# plt.show()

# plt.scatter(y_test, y_pred)
# plt.title('Actual vs Predicted Values')
# plt.xlabel('Actual')
# plt.ylabel('Predicted')
# plt.show()

In [19]:
# poly = PolynomialFeatures(degree = 3)
# X_poly = poly.fit_transform(X_train)

# # print(X_poly.shape)

# poly.fit(X_poly, y_train)
# poly_model = LinearRegression().fit(X_poly, y_train)
# X_poly_test = poly.fit_transform(X_test)

# y_pred = poly_model.predict(X_poly_test)

# print("Polynomial Regression: degree = 3")
# print("R-squared = ", r2_score(y_test, y_pred))
# print("MAE = ",mean_absolute_error(y_test, y_pred))
# print("MSE = ", mean_squared_error(y_test, y_pred))

In [20]:
# visualizer = ResidualsPlot(poly_model, hist=False)

# visualizer.fit(X_poly, y_train)  
# visualizer.score(X_poly_test, y_test)  
# visualizer.show()


# # y_pred = pd.DataFrame(y_pred, columns = ['price'])
# residuals = y_test - y_pred
# sns.histplot(residuals, kde=True)
# plt.title("Residual Count")
# plt.show()

# plt.scatter(y_test, y_pred)
# plt.title('Actual vs Predicted Values')
# plt.xlabel('Actual')
# plt.ylabel('Predicted')
# plt.show()

In [21]:
# from sklearn.linear_model import Ridge
# from sklearn.linear_model import RidgeCV
# from sklearn.model_selection import RepeatedKFold
# from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
# from loguniform import LogUniform

# model = Ridge()
# # cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=18)

# space = dict()
# space['solver'] = ['svd', 'cholesky', 'lsqr', 'sag']
# space['alpha'] = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 5, 10, 100]
# space['fit_intercept'] = [True, False]
# space['normalize'] = [True, False]

# search = GridSearchCV(model, space, n_jobs=-1, cv=5)

# result = search.fit(X_train, y_train)

# print('Best Score: %s' % result.best_score_)
# print('Best Hyperparameters: %s' % result.best_params_)

In [22]:
# ridge = Ridge(alpha = 1, solver='sag')
# ridge.fit(X_train, y_train)
# y_pred = ridge.predict(X_test)

# print("Ridge Regression: alpha = 1")
# print("R-squared = ", r2_score(y_test, y_pred))
# print("MAE = ",mean_absolute_error(y_test, y_pred))
# print("MSE = ", mean_squared_error(y_test, y_pred))

In [23]:
# visualizer = ResidualsPlot(ridge, hist=False)

# visualizer.fit(X_train, y_train)  
# visualizer.score(X_test, y_test)  
# visualizer.show()


# # y_pred = pd.DataFrame(y_pred, columns = ['price'])
# residuals = y_test - y_pred
# sns.histplot(residuals, kde=True)
# plt.title("Residual Count")
# plt.show()

# plt.scatter(y_test, y_pred)
# plt.title('Actual vs Predicted Values')
# plt.xlabel('Actual')
# plt.ylabel('Predicted')
# plt.show()

In [24]:
# #Ridge Regression

# ridge = Ridge(alpha = 5, solver='sag')
# ridge.fit(X_train, y_train)
# y_pred = ridge.predict(X_test)

# print("Ridge Regression: alpha = 5")
# print("R-squared = ", r2_score(y_test, y_pred))
# print("MAE = ",mean_absolute_error(y_test, y_pred))
# print("MSE = ", mean_squared_error(y_test, y_pred))
# # print('done')

In [25]:
# visualizer = ResidualsPlot(ridge, hist=False)

# visualizer.fit(X_train, y_train)  
# visualizer.score(X_test, y_test)  
# visualizer.show()

# # y_pred = pd.DataFrame(y_pred, columns = ['price'])

# residuals = y_test - y_pred
# sns.histplot(residuals, kde=True)
# plt.title("Residual Count")
# plt.show()

# plt.scatter(y_test, y_pred)
# plt.title('Actual vs Predicted Values')
# plt.xlabel('Actual')
# plt.ylabel('Predicted')
# plt.show()

In [26]:
# ridge = Ridge(alpha = .001, solver='lsqr', normalize = True)
# ridge.fit(X_train, y_train)
# y_pred = ridge.predict(X_test)

# print("Ridge Regression: alpha = .001")
# print("R-squared = ", r2_score(y_test, y_pred))
# print("MAE = ",mean_absolute_error(y_test, y_pred))
# print("MSE = ", mean_squared_error(y_test, y_pred))

In [27]:
# visualizer = ResidualsPlot(ridge, hist=False)

# visualizer.fit(X_train, y_train)  
# visualizer.score(X_test, y_test)  
# visualizer.show()
# # print(y_test)
# # print(y_pred)

# # print(type(y_test))
# # print(type(y_pred))

# # y_pred = pd.DataFrame(y_pred, columns = ['price'])

# residuals = y_test - y_pred
# # print(residuals)
# sns.histplot(residuals, kde=True)
# plt.title("Residual Count")
# plt.show()

# plt.scatter(y_test, y_pred)
# plt.title('Actual vs Predicted Values')
# plt.xlabel('Actual')
# plt.ylabel('Predicted')
# plt.show()

In [28]:

# from sklearn.linear_model import Lasso
# model = Lasso()
# # cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=18)

# space = dict()
# # space['solver'] = ['svd', 'cholesky', 'lsqr', 'sag']
# space['alpha'] = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 5, 10, 100]
# space['max_iter'] = [100, 1000, 10000]
# space['fit_intercept'] = [True, False]
# space['normalize'] = [True, False]

# search = GridSearchCV(model, space, n_jobs=-1, cv=5)

# result = search.fit(X_train, y_train)

# print('Best Score: %s' % result.best_score_)
# print('Best Hyperparameters: %s' % result.best_params_)

In [29]:
# lasso = Lasso(alpha = 1, max_iter=10000, normalize = False)
# # lasso.fit(X_train, y_train)

# # lasso.set_params(alpha=lassocv.alpha_)
# lasso.fit(X_train, y_train)
# # print(lasso.coef_)
# y_pred = lasso.predict(X_test)
# print("Lasso Regression: alpha = 1")
# print("R-squared = ", r2_score(y_test, y_pred))
# print("MAE = ",mean_absolute_error(y_test, y_pred))
# print("MSE = ", mean_squared_error(y_test, y_pred))

In [30]:
# y_pred = y_pred.reshape(904,1)
# residuals = y_test-y_pred

# plt.scatter(y_pred, residuals, color='blue')
# # plt.plot(0,0, color='black')
# plt.title('Residuals for Lasso')
# plt.xlabel('Predicted Value')
# plt.ylabel('Residuals')
# plt.show()

# sns.histplot(residuals, kde=True)
# plt.title("Residual Count")
# plt.show()

# plt.scatter(y_test, y_pred)
# plt.title('Actual vs Predicted Values')
# plt.xlabel('Actual')
# plt.ylabel('Predicted')
# plt.show()

In [31]:
# #Lasso Regression
# lasso = Lasso(alpha = .001, max_iter=1000, normalize = False)

# lasso.fit(X_train, y_train)
# # print(lasso.coef_)
# y_pred = lasso.predict(X_test)
# print("Lasso Regression: alpha = .001")
# print("R-squared = ", r2_score(y_test, y_pred))
# print("MAE = ",mean_absolute_error(y_test, y_pred))
# print("MSE = ", mean_squared_error(y_test, y_pred))

In [32]:
# # y_pred = pd.DataFrame(y_pred, columns = ['price'])
# y_pred = y_pred.reshape(904,1)
# residuals = y_test - y_pred
# # sns.histplot(residuals, kde=True)

# plt.scatter(y_pred, residuals, color='blue')
# plt.title('Residuals for Lasso')
# plt.xlabel('Predicted Value')
# plt.ylabel('Residuals')
# plt.show()


# sns.histplot(residuals, kde=True)
# plt.title("Residual Count")
# plt.show()

# plt.scatter(y_test, y_pred)
# plt.title('Actual vs Predicted Values')
# plt.xlabel('Actual')
# plt.ylabel('Predicted')
# plt.show()

In [33]:
# lasso = Lasso(alpha = 5, max_iter=1000, normalize = False)

# lasso.fit(X_train, y_train)
# y_pred = lasso.predict(X_test)
# print("Lasso Regression: alpha = 5")
# print("R-squared = ", r2_score(y_test, y_pred))
# print("MAE = ",mean_absolute_error(y_test, y_pred))
# print("MSE = ", mean_squared_error(y_test, y_pred))

In [34]:
# # y_pred = pd.DataFrame(y_pred, columns = ['price'])
# y_pred = y_pred.reshape(904,1)
# residuals = y_test - y_pred


# plt.scatter(y_pred, residuals, color='blue')
# plt.title('Residuals for Lasso')
# plt.xlabel('Predicted Value')
# plt.ylabel('Residuals')
# plt.show()

# sns.histplot(residuals, kde=True)
# plt.title("Residual Count")
# plt.show()

# plt.scatter(y_test, y_pred)
# plt.title('Actual vs Predicted Values')
# plt.xlabel('Actual')
# plt.ylabel('Predicted')
# plt.show()
